<a href="https://colab.research.google.com/github/NatashaKamami/Music-Recommendation-System/blob/main/Getting_the_song_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Getting the data**

Due to limitations when it came to scraping song data directly from Spotify using their web API, the data I decided to use is a combination of 2 kaggle datasets which had song data with all the features i needed for my recommender system (song names, artist names, song audio features and song popularity scores.)

Links to the datasets:
- https://www.kaggle.com/datasets/maharshipandya/-spotify-tracks-dataset

- https://www.kaggle.com/datasets/amitanshjoshi/spotify-1million-tracks


In [ ]:
import pandas as pd

data1 = pd.read_csv('/content/spotify_data.csv')
data2 = pd.read_csv('/content/dataset.csv')

##**Extracting songs with the highest popularity from the 2 datasets (<=70) and merging them**

In [ ]:
# Filter both datasets to include only songs with popularity of 70 and higher
data1_filtered = data1[(data1["popularity"] >= 70)]
data2_filtered = data2[(data2["popularity"] >= 70)]


In [ ]:
# Rename columns to match between datasets
data2_filtered = data2_filtered.rename(columns={
    "artists": "artist_name",
    "album_name": "album",
    "track_genre": "genre"
})

In [ ]:
# Select only the matching columns in both datasets
common_columns = [
    "track_id", "artist_name", "track_name", "popularity", "duration_ms",
    "danceability", "energy", "key", "loudness", "mode", "speechiness",
    "acousticness", "instrumentalness", "liveness", "valence", "tempo",
    "time_signature", "genre"
]

data1_filtered = data1_filtered[common_columns]
data2_filtered = data2_filtered[common_columns]

In [ ]:
# Merge the datasets
songs_data = pd.concat([data1_filtered, data2_filtered], ignore_index=True)

# **Handling duplicates**

In [ ]:
# Dropping exact duplicates
songs_data = songs_data.drop_duplicates()

In [ ]:
# Drop duplicates with the same track_name while keeping the first occurrence of each song
songs_data = songs_data.sort_values(by=['popularity'], ascending=False)
songs_data.drop_duplicates(subset=['track_name'], keep='first', inplace=True)

In [ ]:
# Drop duplicates with the same track_id while keeping the first occurrence of each song
songs_data = songs_data.drop_duplicates(subset='track_id', keep='first')

# Reset the index after dropping duplicates
songs_data = songs_data.reset_index(drop=True)

In [ ]:
# Saving the clean data into a csv file
songs_data.to_csv('song_data.csv', index=False)

In [ ]:
songs_data.head()

,track_id,artist_name,track_name,popularity,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,genre
0,3nqQXoyQOWXiESFLlDF1hG,Sam Smith;Kim Petras,Unholy (feat. Kim Petras),100,156943,0.714,0.472,2,-7.375,1,0.0864,0.01300,0.000005,0.2660,0.238,131.121,4,pop
1,0yLdNVWF3Srea0uzk55zFn,Miley Cyrus,Flowers,100,200455,0.707,0.681,0,-4.325,1,0.0668,0.06320,0.000005,0.0322,0.646,117.999,4,pop
2,2tTmW7RDtMQtBk7m2rYeSw,Bizarrap;Quevedo,"Quevedo: Bzrp Music Sessions, Vol. 52",99,198937,0.621,0.782,2,-5.548,1,0.0440,0.01250,0.033000,0.2300,0.550,128.033,4,hip-hop
3,5ww2BF9slyYgNOk37BlC4u,Manuel Turizo,La Bachata,98,162637,0.835,0.679,7,-5.329,0,0.0364,0.58300,0.000002,0.2180,0.850,124.980,4,reggaeton
4,4uUG5RXrOk84mYEfFvj3cK,David Guetta;Bebe Rexha,I'm Good (Blue),98,175238,0.561,0.965,7,-3.673,0,0.0343,0.00383,0.000007,0.3710,0.304,128.040,4,pop


## **Column Description:**

1. **track_id:** The Spotify ID for the track

2. **artist_name:** The artists' names who performed the track. If there is more than one artist, they are separated by a ;

3. **track_name:** Name of the track

5. **popularity:** The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity.

6. **duration_ms:** The track length in milliseconds

7. **danceability:** Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable

9. **energy:** Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale

10. **key:** The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. If no key was detected, the value is -1

11. **loudness:** The overall loudness of a track in decibels (dB)

12. **mode:** Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0

13. **speechiness:** Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks

14. **acousticness:** A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic

15. **instrumentalness:** Predicts whether a track contains no vocals. "Ooh" and "aah" sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly "vocal". The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content

16. **liveness:** Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live

17. **valence:** A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry)

18. **tempo:** The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration

19. **time_signature:** An estimated time signature. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). The time signature ranges from 3 to 7 indicating time signatures of 3/4, to 7/4.

20. **genre:** The genre in which the track belongs